In [96]:
import pandas as pd
import numpy as np

import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline

In [97]:
ttf_weekly_eur_mwh = pd.read_excel(
    "../data/gas_future/raw/statistic_id1267202_weekly-dutch-ttf-gas-prices-2021-2022.xlsx", sheet_name="Data", skiprows=3, parse_dates=True
).drop("Date", axis=1).set_index("DateFixed").resample("1W").ffill()
ttf_weekly_eur_mwh.head(5)

,Price
DateFixed,
2021-01-10,17.89
2021-01-17,18.43
2021-01-24,17.35
2021-01-31,17.94
2021-02-07,17.72


In [98]:
ttf_weekly_eur_mwh.hvplot()

:Curve   [DateFixed]   (Price)

In [99]:
ttf_daily_dollar_million_btu = pd.read_csv("../data/gas_future/raw/ODA-PNGASEU_USD.csv", parse_dates=True, index_col=0).resample("D").ffill()
ttf_daily_dollar_million_btu.head(5)

,Value
Date,
1985-01-31,3.81
1985-02-01,3.81
1985-02-02,3.81
1985-02-03,3.81
1985-02-04,3.81


In [100]:
ttf_daily_dollar_million_btu.hvplot()

:Curve   [Date]   (Value)

In [101]:
min_ttf_usd_date = ttf_daily_dollar_million_btu.index.min()
min_ttf_usd_date

Timestamp('1985-01-31 00:00:00', freq='D')

In [102]:
eur_usd_exchange = pd.read_csv("../data/misc/eur_usd.csv", skiprows=5, parse_dates=True, index_col=0)
eur_usd_exchange.head(5)

,value,status
date,,
2022-01-28,1.1138,Normal value (A)
2022-01-27,1.1160,Normal value (A)
2022-01-26,1.1277,Normal value (A)
2022-01-25,1.1268,Normal value (A)
2022-01-24,1.1304,Normal value (A)


In [103]:
eur_usd_ex_clean = eur_usd_exchange[eur_usd_exchange["status"] != "Missing value; holiday or weekend (H)"].drop("status", axis=1).resample("1D").ffill()
eur_usd_ex_clean["exchange_value"] = eur_usd_ex_clean["value"].astype(np.float64)
eur_usd_ex_clean = eur_usd_ex_clean.drop("value", axis=1)
eur_usd_ex_clean.hvplot()

:Curve   [date]   (exchange_value)

In [104]:
ttf_daily_dollarmbtu_exchange = pd.merge(ttf_daily_dollar_million_btu, eur_usd_ex_clean, left_index=True, right_index=True, how="left").dropna().resample("1D").ffill()
ttf_daily_dollarmbtu_exchange["Value"] = ttf_daily_dollarmbtu_exchange["Value"] / ttf_daily_dollarmbtu_exchange["exchange_value"]
ttf_daily_eurmbtu_exchange = ttf_daily_dollarmbtu_exchange.drop("exchange_value", axis=1)
ttf_daily_eurmbtu_exchange.hvplot()

:Curve   [Date]   (Value)

In [105]:
# 1 Million BTU is equal to 0.293071 MWh
ttf_daily_eurmbtu_exchange["Value"] = ttf_daily_eurmbtu_exchange["Value"] / 0.293071
ttf_weekly_eur_mwh_calculated = ttf_daily_eurmbtu_exchange.resample("1W").mean()
ttf_weekly_eur_mwh_calculated.hvplot()

:Curve   [Date]   (Value)

In [108]:
ttf_weekly_eur_mwh_merged = pd.merge(ttf_weekly_eur_mwh, ttf_weekly_eur_mwh_calculated, left_index=True, right_index=True, how="outer")
ttf_weekly_eur_mwh_merged.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [114]:
ttf_weekly_eur_mwh_combined = pd.DataFrame({"price": ttf_weekly_eur_mwh_merged["Price"].combine_first(ttf_weekly_eur_mwh_merged["Value"])})
ttf_weekly_eur_mwh_combined.hvplot()

:Curve   [index]   (price)

In [115]:
ttf_weekly_eur_mwh_combined.to_csv("../data/gas_future/ttf_weekly_eur_mwh_combined.csv")